In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-feb-2022/train.csv
/kaggle/input/tabular-playground-series-feb-2022/test.csv


In [2]:
df = pd.read_csv('../input/tabular-playground-series-feb-2022/train.csv')
df.head()

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,0,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Streptococcus_pyogenes
1,1,-9.536743e-07,-0.000010,-0.000043,0.000886,-0.000200,0.000760,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,0.000914,0.000914,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Salmonella_enterica
2,2,-9.536743e-07,-0.000002,0.000007,0.000129,0.000268,0.000270,0.000243,0.000125,0.000001,...,0.000084,0.000048,0.000081,0.000106,0.000072,0.000010,0.000008,0.000019,1.046326e-06,Salmonella_enterica
3,3,4.632568e-08,-0.000006,0.000012,0.000245,0.000492,0.000522,0.000396,0.000197,-0.000003,...,0.000151,0.000100,0.000180,0.000202,0.000153,0.000021,0.000015,0.000046,-9.536743e-07,Salmonella_enterica
4,4,-9.536743e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-0.000010,-9.536743e-07,Enterococcus_hirae


In [3]:
df.columns

Index(['row_id', 'A0T0G0C10', 'A0T0G1C9', 'A0T0G2C8', 'A0T0G3C7', 'A0T0G4C6',
       'A0T0G5C5', 'A0T0G6C4', 'A0T0G7C3', 'A0T0G8C2',
       ...
       'A8T0G1C1', 'A8T0G2C0', 'A8T1G0C1', 'A8T1G1C0', 'A8T2G0C0', 'A9T0G0C1',
       'A9T0G1C0', 'A9T1G0C0', 'A10T0G0C0', 'target'],
      dtype='object', length=288)

In [4]:
df['target'].nunique()

10

In [5]:
cat_col = list( set(df.columns ) - set(df._get_numeric_data().columns))
cat_col

['target']

In [6]:
col1 = [col for col in df.columns if df[col].nunique()<=25]
col1

['A0T0G0C10',
 'A0T0G1C9',
 'A0T0G9C1',
 'A0T0G10C0',
 'A0T1G0C9',
 'A0T10G0C0',
 'A1T0G9C0',
 'A10T0G0C0',
 'target']

In [7]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(df['target'])
df['target'] = encoder.transform(df.target)

In [8]:
d = dict(zip(encoder.transform(encoder.classes_),encoder.classes_))
print(d)

{0: 'Bacteroides_fragilis', 1: 'Campylobacter_jejuni', 2: 'Enterococcus_hirae', 3: 'Escherichia_coli', 4: 'Escherichia_fergusonii', 5: 'Klebsiella_pneumoniae', 6: 'Salmonella_enterica', 7: 'Staphylococcus_aureus', 8: 'Streptococcus_pneumoniae', 9: 'Streptococcus_pyogenes'}


In [9]:
df['target']

0         9
1         6
2         6
3         6
4         2
         ..
199995    6
199996    9
199997    8
199998    7
199999    5
Name: target, Length: 200000, dtype: int64

In [10]:
xgb_params = { 
                         'n_jobs' : 4,
                         'n_estimators' : 2000,
                         'tree_method' : 'gpu_hist',
                         'learning_rate': 0.1,
                         'subsample' : 0.9,
                         'max_depth' : 3,
                         'colsample_bytree' : 0.5,
                         'reg_alpha' : 30,
                         'eval_metric' : 'rmse',
    'min_child_weight' : 1
             }

In [11]:
feats = [col for col in df.columns if col != 'target' and col !='row_id']
# from xgboost import XGBClassifier
# model = XGBClassifier(**xgb_params)
# x = df[feats]
# y = df['target']

In [12]:
from sklearn.model_selection import StratifiedKFold
fold = StratifiedKFold(n_splits = 11,shuffle = True)
test = pd.read_csv('../input/tabular-playground-series-feb-2022/test.csv',index_col='row_id')

In [13]:
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
y_pred = []
scores = []
X = df[feats]
y = pd.DataFrame(pd.DataFrame(encoder.fit_transform(df['target']), columns=['target']).values.ravel())

for _fold,(train_idx,test_idx) in enumerate(fold.split(X,y)):
    print(f"FOLD:{_fold+1}")
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx].values.ravel()
    X_valid, y_valid = X.iloc[test_idx], y.iloc[test_idx].values.ravel()
    model = model = CatBoostClassifier(iterations=1000,
                           task_type="GPU",
                           devices='0:1',
                                      max_depth=8,
                                        verbose = 750,
                                      learning_rate=0.09)
    model.fit(X_train, y_train)
    valid_pred = model.predict(X_valid)
    valid_score = accuracy_score(y_valid, valid_pred)
    print(f'Accuracy score: {valid_score:6f}\n')
    scores.append(valid_score)
    y_pred.append(model.predict(test))

FOLD:1
0:	learn: 1.9871774	total: 63.9ms	remaining: 1m 3s
750:	learn: 0.0779052	total: 33.8s	remaining: 11.2s
999:	learn: 0.0556468	total: 44.3s	remaining: 0us
Accuracy score: 0.992245

FOLD:2
0:	learn: 1.9880968	total: 57.8ms	remaining: 57.7s
750:	learn: 0.0782768	total: 33.7s	remaining: 11.2s
999:	learn: 0.0571802	total: 44.8s	remaining: 0us
Accuracy score: 0.989770

FOLD:3
0:	learn: 1.9939065	total: 54.9ms	remaining: 54.8s
750:	learn: 0.0792704	total: 33.5s	remaining: 11.1s
999:	learn: 0.0579657	total: 43.8s	remaining: 0us
Accuracy score: 0.990870

FOLD:4
0:	learn: 1.9876461	total: 59.1ms	remaining: 59.1s
750:	learn: 0.0770758	total: 33.8s	remaining: 11.2s
999:	learn: 0.0566530	total: 44.3s	remaining: 0us
Accuracy score: 0.990595

FOLD:5
0:	learn: 1.9891377	total: 62.9ms	remaining: 1m 2s
750:	learn: 0.0794972	total: 33.8s	remaining: 11.2s
999:	learn: 0.0578379	total: 44.1s	remaining: 0us
Accuracy score: 0.990650

FOLD:6
0:	learn: 1.9874125	total: 57.7ms	remaining: 57.6s
750:	learn: 

In [14]:
from scipy.stats import mode
y_pred2 = y_pred
y_pred2 = mode(y_pred2).mode[0]
y_pred2

array([[4],
       [6],
       [2],
       ...,
       [0],
       [0],
       [9]])

In [15]:

y_pred2 = encoder.inverse_transform(y_pred2)
y_pred2
preds=[]

print(d)
for i in y_pred2:
    preds.append(d[i])
    

{0: 'Bacteroides_fragilis', 1: 'Campylobacter_jejuni', 2: 'Enterococcus_hirae', 3: 'Escherichia_coli', 4: 'Escherichia_fergusonii', 5: 'Klebsiella_pneumoniae', 6: 'Salmonella_enterica', 7: 'Staphylococcus_aureus', 8: 'Streptococcus_pneumoniae', 9: 'Streptococcus_pyogenes'}


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [16]:
#preds

In [17]:
score = np.array(scores).mean()
print(f'Mean accuracy score: {score:6f}')



y_pred = mode(y_pred).mode[0]
y_pred = encoder.inverse_transform(y_pred)



Mean accuracy score: 0.990875


In [18]:
y_pred

array([4, 6, 2, ..., 0, 0, 9])

In [19]:

sample = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')

sample['target'] = preds
sample.to_csv('submission.csv', index=False)

In [20]:
sample.head()

,row_id,target
0,200000,Escherichia_fergusonii
1,200001,Salmonella_enterica
2,200002,Enterococcus_hirae
3,200003,Salmonella_enterica
4,200004,Staphylococcus_aureus
